In [61]:
import os 
import json
import pprint
import tweepy

# Load Twitter API credentials from external file

Create a json file named `twitter_credentials.json`

```
{
    "api_key": "...",
    "api_secret": "...",
    "access_token": "...",
    "access_secret": "...",
    "bearer_token": "..."
}
```

In [62]:
with open('twitter_credentials.json') as f:
   api_data = json.load(f)

api_key = api_data['api_key']
api_secret = api_data['api_secret']
access_token = api_data['access_token']
access_secret = api_data['access_secret']
bearer_token = api_data['bearer_token']

In [63]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


In [53]:
accounts = ["mizzaro", "Miccighel_"]

data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

def get_followers(user, quantity):
    
    print(f"Processing user {user}")
    
    followers_of_user = []
    
    for item in tweepy.Cursor(
        api.followers, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
    ).items():
        
        json_data = item._json
        
        found_follower = {}
        found_follower["id"] = json_data["id"]
        found_follower["name"] = json_data["name"]
        found_follower["screen_name"] = json_data["screen_name"]
        found_follower["location"] = json_data["location"]
        found_follower["description"] = json_data["description"]
        found_follower["protected"] = json_data["protected"]
        found_follower["followers_count"] = json_data["followers_count"]
        found_follower["friends_count"] = json_data["friends_count"]
        
        if (found_follower not in followers_of_user) and\
            not found_follower["protected"] and\
            (found_follower["followers_count"] < 1000) and\
            (found_follower["friends_count"] < 1000):
            followers_of_user.append(found_follower)

        if len(followers_of_user) == quantity:
            break
            
    print(f"Found {len(followers_of_user)} followers for user {user}")

    return followers_of_user


def get_friends(user, quantity):
    
    print(f"Processing user {user}")
    
    friends_of_user = []
    
    for item in tweepy.Cursor(
        api.friends, 
        screen_name=user, 
        skip_status=True, 
        include_user_entities=False
    ).items():
        
        json_data = item._json
        
        found_friend = {}
        found_friend["id"] = json_data["id"]
        found_friend["name"] = json_data["name"]
        found_friend["screen_name"] = json_data["screen_name"]
        found_friend["location"] = json_data["location"]
        found_friend["description"] = json_data["description"]
        found_friend["protected"] = json_data["protected"]
        found_friend["friends_count"] = json_data["friends_count"]
        found_friend["followers_count"] = json_data["followers_count"]
        
        # append only when following/follower < 1000
        if (found_friend not in friends_of_user) and\
            not found_friend["protected"] and\
           (found_friend["friends_count"] < 1000) and\
            (found_friend["followers_count"] < 1000):
            friends_of_user.append(found_friend)

        # exit when friends == quantity
        if len(friends_of_user) == quantity:
            break
            
    print(f"Found {len(friends_of_user)} friends of user {user}")
    
    return friends_of_user

def get_tweets(user, quantity):

    tweets = []
    for item in tweepy.Cursor(
        api.user_timeline, 
        screen_name=user,
        tweet_mode="extended"
    ).items(quantity):
        raw_tweet = item._json
        tweet = {}
        tweet["id"]= raw_tweet["id"]
        tweet["created_at"] = raw_tweet["created_at"]
        tweet["full_text"] = raw_tweet["full_text"],
        tweet["favorite_count"]= raw_tweet["favorite_count"]
        tweets.append(tweet)
    return tweets

# First level followers

In [51]:

first_level_followers = {}
for account in accounts:
    first_level_followers[account] = get_followers(account, 5)
    
serialize_json(data_folder, "first_level_followers.json", first_level_followers)

Processing user mizzaro
Found 5 followers for user mizzaro
Processing user Miccighel_
Found 5 followers for user Miccighel_
Data serialized to path: data/first_level_followers.json


# First level friends

In [54]:

first_level_friends = {}
for account in accounts:
    first_level_friends[account] = get_friends(account, 5)
    
serialize_json(data_folder, "first_level_friends.json", first_level_friends)

Processing user mizzaro
Found 5 friends of user mizzaro
Processing user Miccighel_
Found 5 friends of user Miccighel_
Data serialized to path: data/first_level_friends.json


# Second level followers

In [55]:
second_level_followers = {}
first_level_followers_data = read_json(f"{data_folder}/first_level_followers.json")

for followers_list in first_level_followers_data.values():
    for follower in followers_list:
        second_level_followers[follower['screen_name']] = get_followers(follower['screen_name'], 5)
    
serialize_json(data_folder, "second_level_followers.json", second_level_followers)

Data read from path: data/first_level_followers.json
Processing user OlegZendel
Found 5 followers for user OlegZendel
Processing user LyndaSaidLhadj
Found 5 followers for user LyndaSaidLhadj
Processing user acai2021
Found 5 followers for user acai2021
Processing user rodger_benham
Found 5 followers for user rodger_benham
Processing user ColoursRiot
Found 5 followers for user ColoursRiot
Processing user Il_Bale_
Found 1 followers for user Il_Bale_
Processing user Outsider289
Found 5 followers for user Outsider289
Processing user mizzaro
Found 5 followers for user mizzaro
Processing user EUROARTISTS
Found 5 followers for user EUROARTISTS
Processing user fra_bomba6
Found 4 followers for user fra_bomba6
Data serialized to path: data/second_level_followers.json


# Second level friends

In [56]:
second_level_friends = {}
first_level_friends_data = read_json(f"{data_folder}/first_level_friends.json")

for friends_list in first_level_friends_data.values():
    for friends in friends_list:
        second_level_friends[friends['screen_name']] = get_friends(friends['screen_name'], 5)
    
serialize_json(data_folder, "second_level_friends.json", second_level_friends)

Data read from path: data/first_level_friends.json
Processing user MaxZancanaro
Found 5 friends of user MaxZancanaro
Processing user CIRES_AU
Found 5 friends of user CIRES_AU
Processing user mccart_richard
Found 5 friends of user mccart_richard
Processing user Miccighel_
Found 5 friends of user Miccighel_
Processing user rmit_cidda
Found 5 friends of user rmit_cidda
Processing user fra_bomba6
Found 5 friends of user fra_bomba6
Processing user Outsider289
Found 5 friends of user Outsider289
Processing user BiancolinoElisa
Found 5 friends of user BiancolinoElisa
Processing user ZuzanPink
Found 5 friends of user ZuzanPink
Processing user eglu81


Rate limit reached. Sleeping for: 866


Found 5 friends of user eglu81
Data serialized to path: data/second_level_friends.json


# Last 5 tweets of first and second level followers

In [57]:

first_level_followers_data = read_json(f"{data_folder}/first_level_followers.json")
second_level_followers_data = read_json(f"{data_folder}/second_level_followers.json")

for followers_list in first_level_followers_data.values():
    for follower in followers_list:
        if not os.path.exists(f"{data_folder}/{follower['screen_name']}_tweets.json"):
            tweets = get_tweets(follower['screen_name'], 5)
            serialize_json(data_folder, f"{follower['screen_name']}_tweets.json", tweets)


for followers_list in second_level_followers_data.values():
    for follower in followers_list:
        print(follower['screen_name'])
        if not os.path.exists(f"{data_folder}/{follower['screen_name']}_tweets.json"):
            tweets = get_tweets(follower['screen_name'], 5)
            serialize_json(data_folder, f"{follower['screen_name']}_tweets.json", tweets)

Data read from path: data/first_level_followers.json
Data read from path: data/second_level_followers.json
Marwah_k
ivrik
Dars2022_Sympo
ameertawfik
aleksbobic
gael_dias
Nthakur20
AthmaniAmine
iiia_unipd
OthmaniDr
TilburgU_DCA
srj_krishna
gabriellearuta
lesleywilton
sencerdeniz
NFT_Starr777
MaksudaBilkis
jparapar
ameertawfik
Jinxuan16807201
ciscaredda0111
AmbesiAntonio
BrillantVacant
LuvYahBoo24
Martha_Young
mrdanieltaylor1
Miccighel_
Sogno_Danzante
boreale01
nico_peresson
Giulia13568642
Data serialized to path: data/Giulia13568642_recent_tweets.json
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
modosmanagement
Data serialized to path: data/modosmanagement_recent_tweets.json
isabelleGILLOO1
Data serialized to path: data/isabelleGILLOO1_recent_tweets.json
JoziOpera
Data serialized to path: data/JoziOpera_recent_tweets.json
Tenor_Rguez27
Data serialized to path: data/Tenor_Rguez27_recent_tweets.json
MarigonaQerkezi
Data serialized to path: data/MarigonaQerkezi_recent_tweet